## Step 1: Import necessary modules and load configurations

In [1]:
import sys
import json
import os
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    Trainer, 
    TrainingArguments, 
    DataCollatorForSeq2Seq, 
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from datasets import Dataset

# Set environment variable for memory management
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Load configuration settings
from config import TOKENIZER_PATH, MODEL_PATH, PIPELINE_PARAMS, QLORA_PARAMS,ALPACA_LORA_PROMPTS_USER_PROFILE

# Verification
print("Configuration Loaded:")
print("Tokenizer Path:", TOKENIZER_PATH)
print("Model Path:", MODEL_PATH)
print("Pipeline Parameters:", PIPELINE_PARAMS)
print("QLoRA Parameters:", QLORA_PARAMS)
print("Prompt Template:", ALPACA_LORA_PROMPTS_USER_PROFILE)

c:\Users\Trung\anaconda3\envs\torch_recommender\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuration Loaded:
Tokenizer Path: models/hf-frompretrained-download/meta-llama/Meta-Llama-3-8B-Instruct
Model Path: models/hf-frompretrained-downloadmeta-llama/Meta-Llama-3-8B-Instruct
Pipeline Parameters: {'max_length': 2048, 'num_return_sequences': 1, 'temperature': 0.7, 'top_k': 50, 'top_p': 0.95, 'repetition_penalty': 1.2}
QLoRA Parameters: {'lora_r': 8, 'lora_alpha': 16, 'lora_dropout': 0.05, 'lora_target_modules': ['q_proj', 'v_proj'], 'gradient_accumulation_steps': 2, 'lora_num_epochs': 2, 'lora_val_iterations': 100, 'lora_early_stopping_patience': 10, 'lora_lr': 0.0001, 'lora_micro_batch_size': 1}
Prompt Template: {'instruction': "### Instruction:\n You are a recommender system specialized in creating user profiles. Your task is to create a comprehensive user profile and a list of candidate items based on a user's review. The user Profile consists of: 1. Short-term Interests: Examine the user's most recent items along with their personalized descriptions 2. Long-term Prefer

## Step 2: Load and verify training data

In [2]:
data_path = "QLoRa_finetuning/new_candidate_items_with_profile.json"

# Load the training data
with open(data_path, "r") as file:
    training_data = json.load(file)

# Sample a couple of data points to verify format
print("Training Data Sample:", training_data[:2])  # Display first two entries
# Clear cache before loading model
torch.cuda.empty_cache()
# Verify data structure
print("Data Structure Verification:")
for i, sample in enumerate(training_data[:2]):
    assert "User_ID" in sample, f"User_ID missing in sample {i}"
    assert "User_Profile" in sample, f"User_Profile missing in sample {i}"
    assert "Candidate_Items" in sample, f"Candidate_Items missing in sample {i}"
print("Data verification successful!")

Training Data Sample: [{'User_ID': 'AFQQQ5LGNSQUEBGDCYBAZZE5T3DA', 'User_Profile': '"Short-Term Interests": The user recently reviewed products focused on facial skincare, specifically creams and serums targeting anti-aging concerns such as fine lines, wrinkles, and dehydration. They showed interest in natural and organic ingredients, vegan-friendly options, and cruelty-free practices. Their preference leans towards face washes and moisturizers that cater to specific skin concerns like acne-prone, sensitive, and dry skin.\n"Long-Term Preferences": Based on the entirety of their reviews, some common denominators emerge:\n* Focus on natural ingredients, particularly plant-based extracts, essential oils, and herbal remedies\n* Concern for anti-aging and rejuvenation, seeking effective solutions for maintaining healthy-looking skin\n* Interest in sheet masks, especially those with unique features like eye patches and targeted application areas\n* Appreciation for moisturized and hydrated s

## Step 3: Initialize the Tokenizer and Model with Quantization

In [3]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS if not already set

# Set 4-bit quantization configuration for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',  # Use NormalFloat4 for better memory efficiency
    bnb_4bit_use_double_quant=True  # Double quantization for more memory saving
)
torch.cuda.empty_cache()

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto"  # Automatically maps layers to available GPU memory
)
# Load user reviews
reviews_path = "new_data/new_train_output.json"
with open(reviews_path, "r") as file:
    reviews_data = json.load(file)

# Index reviews by user_id for easy matching
reviews_by_user = {entry["user_id"]: entry["reviews"] for entry in reviews_data}

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
c:\Users\Trung\anaconda3\envs\torch_recommender\lib\site-packages\transformers\quantizers\auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.10s/it]


## Step 4: Preprocessing Function to Match Reviews with Profiles

In [4]:
def preprocess_function(profile_sample):
    user_id = profile_sample["User_ID"]
    reviews = reviews_by_user.get(user_id, [])
    review_texts = [f"Product: {review['product_name']}\nRating: {review['rating']}\nTitle: {review['title']}\nReview: {review['text']}\n" 
                    for review in reviews]
    review_texts = review_texts[-10:]
    formatted_reviews = "\n".join(review_texts)
    
    instruction = ALPACA_LORA_PROMPTS_USER_PROFILE['instruction']
    input_text = f"User reviews:\n{formatted_reviews}" if formatted_reviews else "No reviews available for this user."
    output_text = profile_sample["User_Profile"]
    
    full_text = f"\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output_text}"
    return full_text

# Verify preprocessing
print("Preprocessed Sample:", preprocess_function(training_data[0]))


Preprocessed Sample: 
### Instruction:
 You are a recommender system specialized in creating user profiles. Your task is to create a comprehensive user profile and a list of candidate items based on a user's review. The user Profile consists of: 1. Short-term Interests: Examine the user's most recent items along with their personalized descriptions 2. Long-term Preferences: Analyze all reviews and items from the user’s entire history to capture deeper, stable interests that define the user’s lasting preferences. Use this comprehensive historical data to outline consistent themes and inclinations that have remained steady over time. 3. User Profile Summary: Synthesize these findings into a concise profile (max 200 words) that combines insights from both short-term interests and long-term preferences. Use the short-term interests as a query to refine or highlight relevant themes from the long-term history, and provide a cohesive picture of the user’s tastes, typical habits, and potential

## Step 5: Tokenize and Prepare Data

In [5]:
# Tokenization function
def tokenize_function(sample):
    processed_text = preprocess_function(sample)
    tokenized = tokenizer(
        processed_text,
        truncation=True,
        max_length=PIPELINE_PARAMS['max_length'],
        padding="max_length",
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()  # Set labels identical to input_ids
    return tokenized

# LoRA Configuration
lora_config = LoraConfig(
    r=QLORA_PARAMS['lora_r'],
    lora_alpha=QLORA_PARAMS['lora_alpha'],
    lora_dropout=QLORA_PARAMS['lora_dropout'],
    target_modules=QLORA_PARAMS['lora_target_modules'],
    bias="none",
    task_type="CAUSAL_LM"
)

## Step 6: Configure LoRA and Training Parameters

In [6]:
# Apply LoRA configuration to the model
model = get_peft_model(model, lora_config)
batch_size = 16

# Training sizes
training_sizes = [32]

# Loop through different training sizes
for train_size in training_sizes:
    # Split the dataset
    train_data = training_data[:train_size]
    eval_data = training_data[train_size:train_size + int(0.2 * train_size)]  # 20% of training data for evaluation

    # Tokenize datasets
    train_tokenized_data = [tokenize_function(sample) for sample in train_data]
    eval_tokenized_data = [tokenize_function(sample) for sample in eval_data]

    # Convert tokenized data to Dataset format
    train_dataset = Dataset.from_dict({
        "input_ids": [x["input_ids"][0] for x in train_tokenized_data],
        "attention_mask": [x["attention_mask"][0] for x in train_tokenized_data],
        "labels": [x["labels"][0] for x in train_tokenized_data]
    })
    eval_dataset = Dataset.from_dict({
        "input_ids": [x["input_ids"][0] for x in eval_tokenized_data],
        "attention_mask": [x["attention_mask"][0] for x in eval_tokenized_data],
        "labels": [x["labels"][0] for x in eval_tokenized_data]
    })

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"outputs/adapter_test_candidate_items_epoch_{QLORA_PARAMS['lora_num_epochs']}_{train_size}_samples",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=QLORA_PARAMS['gradient_accumulation_steps'],
        num_train_epochs=QLORA_PARAMS['lora_num_epochs'],
        evaluation_strategy="steps",
        eval_steps=QLORA_PARAMS['lora_val_iterations'],
        save_steps=QLORA_PARAMS['lora_val_iterations'],
        logging_steps=10,
        learning_rate=QLORA_PARAMS['lora_lr'],
        save_total_limit=2,
        load_best_model_at_end=False,
        dataloader_pin_memory=False,
        report_to="none",
        fp16=True
    )

    # Data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, padding=True)

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator
    )

    # Clear GPU cache before training
    torch.cuda.empty_cache()

    # Start training
    print(f"Starting training with {train_size} samples.")
    trainer.train()
    adapter_name = f"adapter_test_user_profile_epoch_{QLORA_PARAMS['lora_num_epochs']}_{train_size}_samples"
    # Save the model and tokenizer in separate directories for each training size
    model.save_pretrained(f"outputs/{adapter_name}")
    tokenizer.save_pretrained(f"outputs/{adapter_name}")
    print(f"Model trained with {train_size} samples saved to outputs/{adapter_name}")


c:\Users\Trung\anaconda3\envs\torch_recommender\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training with 32 samples.


 31%|███▏      | 10/32 [07:04<16:27, 44.89s/it]

{'loss': 5.4053, 'grad_norm': 17.300838470458984, 'learning_rate': 7.8125e-05, 'epoch': 0.62}


 62%|██████▎   | 20/32 [16:03<10:47, 53.98s/it]

{'loss': 2.4451, 'grad_norm': 4.829381942749023, 'learning_rate': 5e-05, 'epoch': 1.25}


 94%|█████████▍| 30/32 [25:00<01:48, 54.01s/it]

{'loss': 1.4705, 'grad_norm': 0.9115433692932129, 'learning_rate': 1.8750000000000002e-05, 'epoch': 1.88}


100%|██████████| 32/32 [26:49<00:00, 50.30s/it]


{'train_runtime': 1609.739, 'train_samples_per_second': 0.04, 'train_steps_per_second': 0.02, 'train_loss': 3.0273856967687607, 'epoch': 2.0}
Model trained with 32 samples saved to outputs/adapter_test_user_profile_epoch_2_32_samples


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
from transformers import BitsAndBytesConfig
from config import *


TOKENIZER_PATH = "models/hf-frompretrained-download/meta-llama/Meta-Llama-3-8B-Instruct"
# Load the base model and tokenizer
base_model_path ="models/hf-frompretrained-downloadmeta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token



model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    #quantization_config=bnb_config,
    device_map="auto",
    #torch_dtype=torch.float16,
)

# Load the user profile adapter
train_size = 64  # Adjust based on your adapter's training size
adapter_path_user_profile =f"outputs/adapter_test_user_profile_epoch_{QLORA_PARAMS['lora_num_epochs']}_{train_size}_samples"
adapter_name_user_profile = "user_profile"
model = PeftModel.from_pretrained(model, adapter_path_user_profile, adapter_name=adapter_name_user_profile)

print(f"Loaded user profile adapter: {adapter_name_user_profile}")

# Load the candidate items adapter
adapter_path_candidate_items = f"outputs/adapter_test_candidate_items_epoch_{QLORA_PARAMS['lora_num_epochs']}_{64}_samples"
adapter_name_candidate_items = "candidate_items"
model.load_adapter(adapter_path_candidate_items, adapter_name=adapter_name_candidate_items)

print(f"Loaded candidate items adapter: {adapter_name_candidate_items}")

# Print the list of adapters loaded into the model
print("Adapters loaded in the model:", list(model.peft_config.keys()))

# Ensure the model is in evaluation mode
model.eval()

# Define the text generation function
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=PIPELINE_PARAMS['max_length'],
        do_sample=True,
        temperature=PIPELINE_PARAMS['temperature'],
        top_k=PIPELINE_PARAMS['top_k'],
        top_p=PIPELINE_PARAMS['top_p'],
        repetition_penalty=PIPELINE_PARAMS['repetition_penalty'],
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Generate user profile
user_reviews = """User reviews:
Product: Moisturizing Facial Emulsion for Restoring Hydrating Smoothing Skin from Manilla Natural Skincare
Rating: 5.0
Title: Intense Hydration
Review: With winter approaching, it's time to start thinking about cold-weather skincare. This moisturizer is a good choice because it's a blend of several skin-loving oils like argan, coconut and macadamia. It's fairly lightweight but it protects and hydrates like a heavier cream. It can be used both day and night. The packaging is pretty enough for display. It can be used by all skin types. I highly recommend this product and the entire line from this brand.<br /><br />Thanks so much for taking the time to read this review. I hope the information that I've provided makes your decision a little easier.

Product: Dr. Denese SkinScience Essential Lipid Anti Aging Power Infusion Dry Oil - Skin Nutrients 97% Organic 100% Natural - Rejuvinating Blend with Amaranth & Resveratrol - Paraben-Free, Cruelty-Free - 2oz
Rating: 5.0
Title: Youth Serum
Review: One of the reasons skin starts to wrinkle as we age is that we make less and less of the natural oil that keeps skin looking youthful. This serum is a good way to supplement what your skin lacks. The oils in this blend are all high quality. The texture is lightweight and easily absorbed. Dr. Denese is a trusted brand when it comes to care for aging skin. Whether you're trying to slow the clock or reverse it a little, this serum should be part of your daily skincare routine. I hope the information that I've provided makes your decision a little easier.

Product: [Abib] Creme coating mask Tone-up solution 17g (5pcs)
Rating: 5.0
Title: Doesn't Leave Skin Sticky
Review: I use lots of sheet masks. Most leave your skin a little sticky when the serum dries. This mask works differently. The sheet locks the moisture in and doesn't leave a sticky residue. It's a great hydration and brightening mask. I highly recommend it.<br /><br />Thanks so much for taking the time to read this review. I hope the information that I've provided makes your decision a little easier.

Product: Easydew 2-Step Face Contour Sheet Mask - Tightening & Anti-Aging & Hydrating Korean Face Mask with DW-EGF, Hyaluronic Acid for Anti-Winkle, Reducing Fine Line & Producing Collagen (Qty 5)
Rating: 5.0
Title: Special Care Mask
Review: I really like that this mask includes special patches for the eyes and smile lines. If you're over 30, you'll appreciate the extra help. You apply the patches first, the the sheet mask. I keep all of my sheet masks in the refrigerator. You'll appreciate the cooling and anti-inflammatory effect if you do so with this mask. I highly recommend this product.<br /><br />Thanks so much for taking the time to read this review. I hope the information that I've provided makes your decision a little easier.

Product: Higher Education Skincare: Goal Digger - Silky Moisturizing Cucumber Creme; formulated for dry and sensitive skin; cucumber extract; natural extract: melon, kale, cabbage, ginger, turmeric - 1.7 fl oz
Rating: 5.0
Title: Lightweight and Creamy
Review: I really like this moisturizer. It's lightweight and creamy. It has a light cucumber scent that I find refreshing. It's a powerhouse at deeply hydrating parched skin. It also locks in moisture to keep skin looking fresh. It contains botanicals that soothe irritated or inflamed skin. It's ideal for those with acne, rosacea or eczema. You can use it as both a day and night cream. I highly recommend this product.<br /><br />Thanks so much for taking the time to read this review. I hope the information that I've provided makes your decision a little easier.

Product: Queendom Unlashed Mascara | Volumizing and Lengthening | Boosts Lash Length | Vegan, Cruelty Free, Paraben Free
Rating: 5.0
Title: New Favorite!!
Review: Every mascara claims to be new and different. This one actually lives up to the hype. The fibers have a novel shape that really makes your lashes POP. If you're not a fan of falsies, this is a great alternative. The formula isn't waterproof, so keep that in mind. I HIGHLY recommend this product.<br /><br />Thanks so much for taking the time to read this review. I hope the information that I've provided makes your decision a little easier.

Product: ABOUT ME MediAnswer Collagen Firming Up Mask 4ea (1box) 25g - 77% Pure Collagen Extract Anti-Aging Total Care Facial Mask Sheet, Powerful Hydrating and Anti-Wrinkle Night Skin Care
Rating: 5.0
Title: Effective Mask
Review: If you're reading this review, you probably already know the anti-aging benefits of collagen. This mask has a high-potency form of collagen along with a special delivery system to help firm your skin. The mask is in two parts, so you can get a better fit than a regular sheet mask. I can't say it performs miracles, but it's definitely worth a try.<br /><br />Thanks so much for taking the time to read this review. I hope the information that I've provided makes your decision a little easier.
"""

prompt_user_profile = (
    ALPACA_LORA_PROMPTS_USER_PROFILE['instruction'] + "\n\n" +
    ALPACA_LORA_PROMPTS_USER_PROFILE['input'].replace("{user_review}", user_reviews)+ "\n### Response"
)

# Set the active adapter to user profile adapter
print(f"\nSetting active adapter to: {adapter_name_user_profile}")
model.set_adapter(adapter_name_user_profile)
print(f"Current active adapter: {model.active_adapter}")

generated_profile = generate_text(prompt_user_profile)
generated_profile = generated_profile[len(prompt_user_profile):].strip()
print("\nGenerated User Profile:")
print(generated_profile)

# Generate candidate items using the generated user profile
prompt_candidate_items = (
    ALPACA_LORA_PROMPTS_CANDIDATE_ITEMS['instruction'] + "\n\n" +
    ALPACA_LORA_PROMPTS_CANDIDATE_ITEMS['input'].replace("{user_profile}", generated_profile) + "\n### Response"
)

# Set the active adapter to candidate items adapter
print(f"\nSetting active adapter to: {adapter_name_candidate_items}")
model.set_adapter(adapter_name_candidate_items)
print(f"Current active adapter: {model.active_adapter}")

generated_items = generate_text(prompt_candidate_items)
generated_items = generated_items[len(prompt_candidate_items):].strip()
print("\nGenerated Candidate Items:")
print(generated_items)


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
c:\Users\Trung\anaconda3\envs\torch_recommender\lib\site-packages\accelerate\utils\modeling.py:1390: UserWarning: Current model requires 1056.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 